In [2]:
#Import 
import glob
import time
import os
import pandas as pd
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
from argparse import ArgumentParser, Namespace
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from itertools import chain
from tqdm import tqdm

I assume that the candidate generation and feature genration has already be run on the training and dev tables

In [3]:
##File paths

# working_path = os.path.abspath('../../contrastive_loss_nn_data')
# working_path = os.path.abspath('../../contrastive_loss_nn_w_embedding_scores')
# working_path = os.path.abspath('../../contrastive_loss_nn_w_embedding_scores_8_12_21')
# working_path = os.path.abspath('../../contrastive_loss_nn_embed_scores_and_values_8_16_21')
# working_path = os.path.abspath('../../contrastive_loss_nn_data_25-75_centroid_folds')
# working_path = os.path.abspath('../../contrastive_loss_nn_w_embedding_scores_9_9_21_v2')
# working_path = os.path.abspath('../../contrastive_loss_nn_train_dev_test_25_75')
# working_path = os.path.abspath('../../SemTabR4_2_0-25_75-v3.1')
working_path = os.path.abspath('../../t2dv2_25_75_train_dev_test-v2')

#GDrive Path: /table-linker-datasets/2019-iswc_challenge_data/t2dv2/canonical-with-context/contrastive_loss_nn_data/t2dv2-train-lof-original-tfidf
# train_path = f'{working_path}/t2dv2-train-lof-original-tfidf/'
# train_path = f'{working_path}/t2dv2-train-lof-original-tfidf-HAS/'
# train_path = f'{working_path}/t2dv2-train-lof-embed_scores_and_values/'
# train_path = f'{working_path}/t2dv2-train-lof-original-tfidf_25-75'
train_path = f'{working_path}/train'

#GDrive Path: /table-linker-datasets/2019-iswc_challenge_data/t2dv2/canonical-with-context/contrastive_loss_nn_data/t2dv2-dev-lof-original-tfidf
# dev_path = f'{working_path}/t2dv2-dev-lof-original-tfidf/'
# dev_path = f'{working_path}/t2dv2-dev-lof-original-tfidf-HAS/'
# dev_path = f'{working_path}/t2dv2-dev-lof-embed_scores_and_values/'
# dev_path = f'{working_path}/sample-t2dv2-train-lof-embed_scores_and_values/'
# dev_path = f'{working_path}/t2dv2-dev-lof-original-tfidf_25-75'
dev_path = f'{working_path}/dev'

test_path = f'{working_path}/test'

fold_dir_name = "fold_1"

features_to_use = ["String_Sim4", "ComplEx"]


In [350]:
train_path = f"{train_path}/{fold_dir_name}/evaluation_rows/"
dev_path = f"{dev_path}/{fold_dir_name}/evaluation_rows/"
test_path = f"{test_path}/{fold_dir_name}/evaluation_rows/"

# Features to use
feature_name_to_cols = {"String_Sim4" : ['monge_elkan','jaro_winkler','levenshtein','retrieval_score'],
                        "String_Sim_w_Context6" : ['monge_elkan','jaro_winkler','levenshtein','retrieval_score',
                                                   'monge_elkan_aliases','singleton'],
                        "Standard7" : ['monge_elkan','jaro_winkler','levenshtein','retrieval_score',
                                       'monge_elkan_aliases','singleton','pagerank'],
                        "Pagerank" : ['pagerank'],
                        "Profile_ComplEx" : ['Profile-ComplEx-embedding-score'],
                        "Profile_TransE" : ['Profile-TransE-embedding-score'],
                        "ComplEx" : ['ComplEx-embedding-score'],
                        "TransE" : ['TransE-embedding-score'],
                        "Profile" : ['profile-score'],
                        "H" : ['H_5x8-embedding-score'],
                        "A" : ['A-embedding-score'],
                        "S" : ['S-embedding-score'],
                        "TFIDF_Profile" : ['TFIDF-Profile-score'],
                        "D_over_.2_Profile" : ['D>.2-Profile-score']
                       }

train_features = [col for feature_name in features_to_use for col in feature_name_to_cols[feature_name]]
features_including_eval_label = train_features if 'evaluation_label' in train_features else train_features + ['evaluation_label']

# **NAME WORKING PATH DESCRIBING FEATURES USED HERE
working_path = f"{working_path}/{'-'.join(features_to_use)}/{fold_dir_name}"

#Note: One would need just the train_path and dev_path to run this notebook. The below files are output after 
# running the notebook.

#GDrive Path: /table-linker-datasets/2019-iswc_challenge_data/t2dv2/canonical-with-context/contrastive_loss_nn_data/training_data/pos_features.pkl
pos_output = f'{working_path}/training_data/pos_features.pkl'
#GDrive Path: /table-linker-datasets/2019-iswc_challenge_data/t2dv2/canonical-with-context/contrastive_loss_nn_data/training_data/neg_features.pkl
neg_output = f'{working_path}/training_data/neg_features.pkl'
#GDrive Path: /table-linker-datasets/2019-iswc_challenge_data/t2dv2/canonical-with-context/contrastive_loss_nn_data/training_data/normalization_factor.pkl
min_max_scaler_path = f'{working_path}/training_data/normalization_factor.pkl'
dev_output_predictions = f'{working_path}/dev_predictions/'
test_output_predictions = f'{working_path}/test_predictions/'
#GDrive Path: /table-linker-datasets/2019-iswc_challenge_data/t2dv2/canonical-with-context/contrastive_loss_nn_data/saved_models
model_save_path = f'{working_path}/saved_models/'

In [351]:
!mkdir -p {dev_output_predictions}
!mkdir -p {test_output_predictions}
!mkdir -p {model_save_path}
!mkdir -p {working_path}/training_data

old 'Specify train features'
The features you specify must be present as columns in the files found in the above specified train / dev paths

In [352]:
# original
# train_features = ['pagerank','retrieval_score','monge_elkan','des_cont_jaccard',
#         'jaro_winkler','levenshtein','singleton','is_lof','num_char','num_tokens',
#        'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
#        'lof-graph-embedding-score', 'lof-reciprocal-rank']

# original plus eval label
# train_features = ['pagerank','retrieval_score','monge_elkan','des_cont_jaccard',
#         'jaro_winkler','levenshtein','singleton','is_lof','num_char','num_tokens',
#        'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
#        'lof-graph-embedding-score', 'lof-reciprocal-rank',
#                   'evaluation_label_copy']

# original plus h, a, s embedding scores (lof-centroid)
# train_features = ['pagerank','retrieval_score','monge_elkan','des_cont_jaccard',
#         'jaro_winkler','levenshtein','singleton','is_lof','num_char','num_tokens',
#        'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
#        'lof-graph-embedding-score', 'lof-reciprocal-rank',
#             'lof-H_5x8-embedding-score', 'lof-A-embedding-score', 'lof-S-embedding-score']

# original plus Profile-ComplEx embedding scores (lof-centroid)
# train_features = ['pagerank','retrieval_score','monge_elkan','des_cont_jaccard',
#         'jaro_winkler','levenshtein','singleton','is_lof','num_char','num_tokens',
#        'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
#        'lof-graph-embedding-score', 'lof-reciprocal-rank',
#                  'lof-Profile-ComplEx-embedding-score']

# original plus Profile-ComplEx embedding scores (lof-centroid) minus other embeddings
# train_features = ['pagerank','retrieval_score','monge_elkan','des_cont_jaccard',
#                   'jaro_winkler','levenshtein','singleton','is_lof','num_char','num_tokens',
#                   'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score', 'lof-reciprocal-rank',
#                   'lof-Profile-ComplEx-embedding-score']

# original minus other embeddings
# train_features = ['pagerank','retrieval_score','monge_elkan','des_cont_jaccard',
#                   'jaro_winkler','levenshtein','singleton','is_lof','num_char','num_tokens',
#                   'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score', 'lof-reciprocal-rank']

# original "standard" features
# train_features = ['pagerank','retrieval_score','monge_elkan','jaro_winkler','levenshtein']

# "standard" features + profile-ComplEx embedding scores
# train_features = ['pagerank','retrieval_score','monge_elkan','jaro_winkler','levenshtein',
#                   'Profile-ComplEx-embedding-score']

# # original "standard" features + profile scores
# train_features = ['pagerank','retrieval_score','monge_elkan','jaro_winkler','levenshtein',
#                   'profile-score']

# # original "standard" + Profile-ComplEx embedding *values*
# train_features = ['pagerank','retrieval_score','monge_elkan','jaro_winkler','levenshtein']
# # TODO - make this neater, no magic numbers
# train_features.extend([f"cand-Profile-ComplEx-emb-dim-{i}" for i in range(100)])
# train_features.extend([f"centroid-Profile-ComplEx-emb-dim-{i}" for i in range(100)])

# just profile-complEx embedding scores
# train_features = ['Profile-ComplEx-embedding-score']

# train_features = ['evaluation_label_copy']

# features_including_eval_label = train_features if 'evaluation_label' in train_features else train_features + ['evaluation_label']


### Generate Training Data

In [353]:
def merge_files(args):
    datapath = args.train_path
    eval_file_names = []
    for (dirpath, dirnames, filenames) in os.walk(datapath):
        for fn in filenames:
            if "csv" not in fn:
                continue
            abs_fn = dirpath + fn
            if not os.path.isfile(abs_fn):
                print(abs_fn)
            assert os.path.isfile(abs_fn)
            if os.path.getsize(abs_fn) == 0:
                continue
            eval_file_names.append(abs_fn)
    df_list = []
    for fn in eval_file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df_list.append(df)
    return pd.concat(df_list) 

def compute_normalization_factor(args, all_data):
    features = train_features
    
    min_max_scaler_path = args.min_max_scaler_path
    all_data_features = all_data[features]
    scaler = MinMaxScaler()
    scaler.fit(all_data_features)
    pickle.dump(scaler, open(min_max_scaler_path, 'wb'))
    return scaler

def generate_train_data(args):
    scaler_path = args.min_max_scaler_path
    scaler = pickle.load(open(scaler_path, 'rb'))
    final_list = []
    
    features = features_including_eval_label
    normalize_features = train_features
    
    evaluation_label = ['evaluation_label']
    positive_features_final = []
    negative_features_final = []
    for i,file in enumerate(tqdm(glob.glob(args.train_path + '/*.csv'))):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
            continue
        print(file_name)
        d_sample = pd.read_csv(file)
        grouped_obj = d_sample.groupby(['column', 'row'])
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            pos_features = []
            neg_features = []
            a = cell[1][cell[1]['evaluation_label'] == 1]
            if a.empty:
                continue
                
            # new code
#             pos_rows = cell[1][(cell[1]['evaluation_label'].astype(int) == 1)][normalize_features].to_numpy()
#             for i in range(len(pos_rows)):
#                 pos_features.append(pos_rows[i])
#             neg_rows = cell[1][(cell[1]['evaluation_label'].astype(int) == -1)][normalize_features].to_numpy()
#             for i in range(min(50,len(neg_rows))):
#                 neg_features.append(neg_rows[i])
#             random.shuffle(pos_features)
#             random.shuffle(neg_features)
#             positive_features_final.append(pos_features)
#             negative_features_final.append(neg_features)
            
#     print(len(positive_features_final), len(positive_features_final[3]))
#     print(len(negative_features_final), len(negative_features_final[3]))
#     pickle.dump(positive_features_final,open(args.pos_output,'wb'))
#     pickle.dump(negative_features_final,open(args.neg_output,'wb'))    
                
                
                
            # -------------------------- OLD CODE ---------------------------
            num_rows = 64
            pos_row = a[features].drop('evaluation_label',axis=1)
            negatives_filtered = cell[1][cell[1]['evaluation_label'] == -1]
            
            #TODO -- MAKE THIS BETTER
            
            negatives_filtered = negatives_filtered[features]
            if 0 in negatives_filtered['evaluation_label'].tolist():
                continue
            if negatives_filtered.empty:
                continue
            neg_list = []
            if num_rows < len(negatives_filtered):
                negatives_filtered = negatives_filtered[negatives_filtered['evaluation_label'] == -1]
#                 top_sample_df = negatives_filtered.sample(n=(num_rows - sum([len(x) for x in neg_list])))
                
                remaining_negatives = negatives_filtered
                for feature in train_features:
                    sorted_by_feature_df = remaining_negatives.sort_values(feature,ascending=False)
                    neg_list.append(sorted_by_feature_df[:2])
                    remaining_negatives = sorted_by_feature_df[2:]
                top_sample_df = remaining_negatives.sample(n=(num_rows - sum([len(x) for x in neg_list])))
    
#                 retrieval_score_df = negatives_filtered.sort_values('retrieval_score',ascending=False)
#                 neg_list.append(retrieval_score_df[:2])
#                 pagerank_score_df = retrieval_score_df[2:].sort_values('pagerank', ascending=False)
#                 neg_list.append(pagerank_score_df[:2])
                
# #                 class_count_score_df = pagerank_score_df[2:].sort_values('lof_class_count_tf_idf_score', ascending=False)
# #                 neg_list.append(class_count_score_df[:2])
# #                 prop_count_score_df = class_count_score_df[2:].sort_values('lof_property_count_tf_idf_score', ascending=False)
# #                 neg_list.append(prop_count_score_df[:2])
                
#                 monge_elkan_score_df = pagerank_score_df[2:].sort_values('monge_elkan', ascending=False)
#                 neg_list.append(monge_elkan_score_df[:2])
#                 jaro_winkler_score_df = monge_elkan_score_df[2:].sort_values('jaro_winkler', ascending=False)
#                 neg_list.append(jaro_winkler_score_df[:2])
# #                 top_sample_df = jaro_winkler_score_df.sample(n=(num_rows - sum([len(x) for x in neg_list])))
                
# #                 graph_embedding_score_df = jaro_winkler_score_df[2:].sort_values('lof-graph-embedding-score', ascending=False)
# #                 neg_list.append(graph_embedding_score_df[:2])
# #                 top_sample_df = graph_embedding_score_df.sample(n=(num_rows - sum([len(x) for x in neg_list])))
                
# #                 h_embedding_score_df = jaro_winkler_score_df[2:].sort_values('lof-H_5x8-embedding-score', ascending=False)
# #                 neg_list.append(h_embedding_score_df[:2])
# #                 a_embedding_score_df = h_embedding_score_df[2:].sort_values('lof-A-embedding-score', ascending=False)
# #                 neg_list.append(a_embedding_score_df[:2])
# #                 s_embedding_score_df = a_embedding_score_df[2:].sort_values('lof-S-embedding-score', ascending=False)
# #                 neg_list.append(s_embedding_score_df[:2])
# #                 top_sample_df = s_embedding_score_df.sample(n=(num_rows - sum([len(x) for x in neg_list])))

# #                 eval_label_df = negatives_filtered.sort_values('evaluation_label_copy', ascending=False)
# #                 neg_list.append(eval_label_df[:2])
# #                 top_sample_df = eval_label_df.sample(n=(num_rows - sum([len(x) for x in neg_list])))
        
# #                 profile_complex_df = eval_label_df[2:].sort_values('lof-Profile-ComplEx-embedding-score', ascending=False)
# #                 neg_list.append(profile_complex_df[:2])
# #                 top_sample_df = profile_complex_df.sample(n=(num_rows - sum([len(x) for x in neg_list])))

#                 profile_complex_df = jaro_winkler_score_df[2:].sort_values('Profile-ComplEx-embedding-score', ascending=False)
#                 neg_list.append(profile_complex_df[:2])
#                 top_sample_df = profile_complex_df.sample(n=(num_rows - sum([len(x) for x in neg_list])))
            
                neg_list.append(top_sample_df)
                top_sample_df = pd.concat(neg_list)
                top_sample_df.drop('evaluation_label', inplace=True, axis=1)
                top_sample_arr = top_sample_df.to_numpy()
            
            for i in range(num_rows):
                neg_features.append(top_sample_arr[i])
            random.shuffle(neg_features)
            for i in range(num_rows):
                pos_row_sample = pos_row.sample(n=1)
                ar = pos_row_sample.to_numpy()
                for ps_ar in ar:
                    pos_features.append(ps_ar)
            positive_features_final.append(pos_features)
            negative_features_final.append(neg_features)
    print(len(positive_features_final), len(positive_features_final[37]))
    print(len(negative_features_final), len(negative_features_final[37]))
    pickle.dump(positive_features_final,open(args.pos_output,'wb'))
    pickle.dump(negative_features_final,open(args.neg_output,'wb'))


In [354]:
gen_training_data_args = Namespace(train_path=train_path, pos_output=pos_output, neg_output=neg_output, 
                 min_max_scaler_path=min_max_scaler_path)
all_data = merge_files(gen_training_data_args)
scaler = compute_normalization_factor(gen_training_data_args, all_data)
generate_train_data(gen_training_data_args)


  0%|          | 0/38 [00:00<?, ?it/s]

58891288_0_1117541047012405958.csv


  3%|▎         | 1/38 [00:01<01:10,  1.92s/it]

39173938_0_7916056990138658530.csv


  8%|▊         | 3/38 [00:03<00:47,  1.37s/it]

...

39107734_2_2329160387535788734.csv


 97%|█████████▋| 37/38 [01:45<00:02,  2.06s/it]

50245608_0_871275842592178099.csv


100%|██████████| 38/38 [01:49<00:00,  2.87s/it]


3831 64
3831 64


### Model Definition

In [355]:
# Dataset
class T2DV2Dataset(Dataset):
    def __init__(self, pos_features, neg_features):
        self.pos_features = pos_features
        self.neg_features = neg_features
    
    def __len__(self):
        return len(self.pos_features)
    
    def __getitem__(self, idx):
        return self.pos_features[idx], self.neg_features[idx]

# Model
class PairwiseNetwork(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        #original 12x24, 24x12, 12x12, 12x1
        self.fc1 = nn.Linear(hidden_size, 2*hidden_size)
        self.fc2 = nn.Linear(2*hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
    
    def forward(self, pos_features, neg_features):
        # Positive pass
        x = F.relu(self.fc1(pos_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pos_out = torch.sigmoid(self.fc4(x))
        
        # Negative Pass
        x = F.relu(self.fc1(neg_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        neg_out = torch.sigmoid(self.fc4(x))
        
        return pos_out, neg_out
    
    def predict(self, test_feat):
        x = F.relu(self.fc1(test_feat))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        test_out = torch.sigmoid(self.fc4(x))
        return test_out

# Pairwise Loss
class PairwiseLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.m = 0
    
    def forward(self, pos_out, neg_out):
        distance = (1 - pos_out) + neg_out
        loss = torch.mean(torch.max(torch.tensor(0), distance))
        return loss

### Training

In [356]:
def generate_dataloader(positive_feat_path, negative_feat_path):
    pos_features = pickle.load(open(positive_feat_path, 'rb'))
    neg_features = pickle.load(open(negative_feat_path, 'rb'))

    pos_features_flatten = list(chain.from_iterable(pos_features))
    neg_features_flatten = list(chain.from_iterable(neg_features))

    train_dataset = T2DV2Dataset(pos_features_flatten, neg_features_flatten)
    train_dataloader = DataLoader(train_dataset, batch_size=64)
    return train_dataloader

def infer_scores(min_max_scaler_path, input_table_path, output_table_path, model):
    scaler = pickle.load(open(min_max_scaler_path, 'rb'))
    
    normalize_features = train_features
    
    for file in glob.glob(input_table_path + '*.csv'):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
            continue
        if file_name == '52299421_0_4473286348258170200.csv':
            continue
        print(file_name)
        d_sample = pd.read_csv(file)
        grouped_obj = d_sample.groupby(['column', 'row'])
        new_df_list = []
        pred = []
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])

            #TODO -- FIGURE OUT HOW TO MAKE THIS CODE NOT DEPEND ON THE CHOSEN FEATURES -- 
            #        CHANGED FROM GRAPH EMBEDDING TO RETRIEVAL SCORE

#             sorted_df = cell[1].sort_values('retrieval_score',ascending=False)
#             sorted_df = cell[1].sort_values('levenshtein',ascending=False)
#             sorted_df = cell[1].sort_values('Profile-ComplEx-embedding-score',ascending=False)
            sorted_df = cell[1]
            sorted_df_features = sorted_df[normalize_features]
            new_df_list.append(sorted_df)
            arr = sorted_df_features.to_numpy()
            test_inp = []
            for a in arr:
                test_inp.append(a)
            test_tensor = torch.tensor(test_inp).float()
            scores = model.predict(test_tensor)
            # new code
            scores_list = torch.squeeze(scores).tolist()
            if not type(scores_list) is list:
                pred.append(scores_list)
            else:
                pred.extend(scores_list)
            #---OLD CODE---
#             pred.extend(torch.squeeze(scores).tolist())
        test_df = pd.concat(new_df_list)
        test_df['siamese_pred'] = pred
        test_df.to_csv(os.path.join(output_table_path, file_name), index=False)

def train(args):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    
    else:
        device = torch.device('cpu')
    train_dataloader = generate_dataloader(args.positive_feat_path, args.negative_feat_path)
    criterion = PairwiseLoss()
    EPOCHS = args.num_epochs
    model = PairwiseNetwork(len(train_features)).to(device=device)
    optimizer = Adam(model.parameters(), lr=args.lr)
    top1_max_prec = 0
    for epoch in range(EPOCHS):
        train_epoch_loss = 0
        avg_loss = 0
        model.train()
        for bid, batch in tqdm(enumerate(train_dataloader), position=0, leave=True):
            positive_feat = torch.tensor(batch[0].float())
            negative_feat = torch.tensor(batch[1].float())
            optimizer.zero_grad()
            pos_out, neg_out = model(positive_feat, negative_feat)
            loss = criterion(pos_out, neg_out)
            loss.backward()
            optimizer.step()
            train_epoch_loss += loss
        avg_loss = train_epoch_loss / bid

        # Evaluation
        model.eval()
        infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
        eval_data = merge_eval_files(args.dev_output)
        res, candidate_eval_data = parse_eval_files_stats(eval_data, 'siamese_pred')
        top1_precision = res['expected_num_tasks_with_model_score_top_one_accurate']/res['num_tasks_with_gt']
        if top1_precision > top1_max_prec:
            top1_max_prec = top1_precision
            model_save_name = 'epoch_{}_loss_{}_top1_{}.pth'.format(epoch, avg_loss, top1_max_prec)
            best_model_path = os.path.join(args.model_save_path, model_save_name)
            torch.save(model.state_dict(), best_model_path)
        
        print("Epoch {}, Avg Loss is {}, epoch top1 {}, max top1 {}".format(epoch, avg_loss, top1_precision, top1_max_prec))
        
    return best_model_path

In [357]:
def merge_eval_files(final_score_path):
    eval_file_names = []
    df_list = []
    for (dirpath, dirnames, filenames) in os.walk(final_score_path):
        for fn in filenames:
            if fn != '52299421_0_4473286348258170200.csv':
                if "csv" not in fn:
                    continue
                abs_fn = os.path.join(dirpath, fn)
                assert os.path.isfile(abs_fn)
                if os.path.getsize(abs_fn) == 0:
                    continue
                eval_file_names.append(abs_fn)
    
    for fn in eval_file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        # df = df.fillna('')
        df_list.append(df)
    return pd.concat(df_list)

def parse_eval_files_stats(eval_data, method):
    table_rows_w_gt_max = {}
    
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'row', 'column'])['table_id'].count().reset_index(name="count")
    res['num_tasks'] = len(eval_data.groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_gt_in_candidate'] = len(eval_data[eval_data['evaluation_label'] == 1].groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_singleton_candidate'] = len(candidate_eval_data[candidate_eval_data['count'] == 1].groupby(['table_id', 'row', 'column']))
    singleton_eval_data = candidate_eval_data[candidate_eval_data['count'] == 1]
    num_tasks_with_singleton_candidate_with_gt = 0
    for i, row in singleton_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) == 1
        if c_e_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_singleton_candidate_with_gt += 1
    res['num_tasks_with_singleton_candidate_with_gt'] = num_tasks_with_singleton_candidate_with_gt
    num_tasks_with_graph_top_one_accurate = []
    num_tasks_with_graph_top_five_accurate = []
    num_tasks_with_graph_top_ten_accurate = []
    expected_num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    has_gt_list = []
    has_gt_in_candidate = []
    # candidate_eval_data = candidate_eval_data[:1]
    for i, row in candidate_eval_data.iterrows():
        #print(i)
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        
        if table_id not in table_rows_w_gt_max:
            table_rows_w_gt_max[table_id] = {}
        if row_idx not in table_rows_w_gt_max[table_id]:
            table_rows_w_gt_max[table_id][row_idx] = 0
        
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
            
        # handle graph-embedding-score
#         s_data = c_e_data.sort_values(by=['lof-graph-embedding-score'], ascending=False)
#         if s_data.iloc[0]['evaluation_label'] == 1:
#             num_tasks_with_graph_top_one_accurate.append(1)
#         else:
#             num_tasks_with_graph_top_one_accurate.append(0)
#         if 1 in set(s_data.iloc[0:5]['evaluation_label']):
#             num_tasks_with_graph_top_five_accurate.append(1)
#         else:
#             num_tasks_with_graph_top_five_accurate.append(0)
#         if 1 in set(s_data.iloc[0:10]['evaluation_label']):
#             num_tasks_with_graph_top_ten_accurate.append(1)
#         else:
#             num_tasks_with_graph_top_ten_accurate.append(0)
        
        #rank on model score

        cands_w_max_score = c_e_data.loc[c_e_data.loc[:,method] == max(c_e_data.loc[:,method])]
        if 1 in cands_w_max_score.loc[:,"evaluation_label"].values:
            table_rows_w_gt_max[table_id][row_idx] = 1
            expected_num_tasks_with_model_score_top_one_accurate.append(1 / len(cands_w_max_score.loc[:,"kg_id"].unique()))
            
        s_data = c_e_data.sort_values(by=[method], ascending=False) 
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)
            
#         cf_e_data = c_e_data.copy()
#         cf_e_data['lof-graph-embedding-score'] = cf_e_data['lof-graph-embedding-score'].replace(np.nan, 0)
#         cf_e_data[method] = cf_e_data[method].replace(np.nan, 0)

#     candidate_eval_data['lof-graph_top_one_accurate'] = num_tasks_with_graph_top_one_accurate
#     candidate_eval_data['lof-graph_top_five_accurate'] = num_tasks_with_graph_top_five_accurate
#     candidate_eval_data['lof-graph_top_ten_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['model_top_one_accurate'] = num_tasks_with_model_score_top_one_accurate
    candidate_eval_data['model_top_five_accurate'] = num_tasks_with_model_score_top_five_accurate
    candidate_eval_data['model_top_ten_accurate'] = num_tasks_with_model_score_top_ten_accurate
    candidate_eval_data['has_gt'] = has_gt_list
    candidate_eval_data['has_gt_in_candidate'] = has_gt_in_candidate
#     res['num_tasks_with_graph_top_one_accurate'] = sum(num_tasks_with_graph_top_one_accurate)
#     res['num_tasks_with_graph_top_five_accurate'] = sum(num_tasks_with_graph_top_five_accurate)
#     res['num_tasks_with_graph_top_ten_accurate'] = sum(num_tasks_with_graph_top_ten_accurate)
    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    res['expected_num_tasks_with_model_score_top_one_accurate'] = sum(expected_num_tasks_with_model_score_top_one_accurate)
#     print(expected_num_tasks_with_model_score_top_one_accurate)
    return res, candidate_eval_data

In [358]:
training_args = Namespace(num_epochs=20, lr=0.001, positive_feat_path=pos_output, negative_feat_path=neg_output,
                         dev_path=dev_path, dev_output=dev_output_predictions,
                         model_save_path=model_save_path, min_max_scaler_path=min_max_scaler_path)

In [359]:
import warnings
warnings.filterwarnings('ignore')

In [360]:
## Call Training
best_model_path = train(training_args)

3831it [00:08, 469.55it/s]


39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


49it [00:00, 487.79it/s]

Epoch 0, Avg Loss is 0.36663055419921875, epoch top1 0.5836614173228346, max top1 0.5836614173228346


3831it [00:11, 328.78it/s]


...

39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


100it [00:00, 493.57it/s]

Epoch 18, Avg Loss is 0.21676358580589294, epoch top1 0.662149631184588, max top1 0.6703701099862517


3831it [00:07, 511.50it/s]


39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv
Epoch 19, Avg Loss is 0.2180793732404709, epoch top1 0.6563080672578704, max top1 0.6703701099862517


## Computing top1 accuracy on dev set with custom code

In [361]:
!ls -lh {model_save_path}

total 96
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:51 epoch_0_loss_0.36663055419921875_top1_0.5836614173228346.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:54 epoch_10_loss_0.2180592119693756_top1_0.670064395359671.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:55 epoch_11_loss_0.21768595278263092_top1_0.6703701099862517.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:52 epoch_1_loss_0.2597721517086029_top1_0.6145508225129447.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:52 epoch_2_loss_0.25142693519592285_top1_0.6282013041609499.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:52 epoch_3_loss_0.24671632051467896_top1_0.6380539788266166.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:53 epoch_4_loss_0.24150140583515167_top1_0.6420870348424641.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:53 epoch_5_loss_0.23349884152412415_top1_0.6481988044734107.pth
-rw-r--r--  1 nicklein  staff   3.3K Sep 10 15:53 epoch_6_loss_0.2299794703722_top1_0.655639280663887.pt

In [363]:
# model = TheModelClass(*args, **kwargs)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model = PairwiseNetwork(len(train_features)).to(device=device)

model.load_state_dict(torch.load("{}".format(best_model_path)))
model.eval()
infer_scores(min_max_scaler_path, dev_path, dev_output_predictions, model)


39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


In [364]:
from sklearn.metrics import f1_score
from tabulate import tabulate
def choose_best_candidates_from_scores(scores):
    choices = []
    for cell_candidate_scores in scores:
        # we need to break ties randomly since the input data may not be formatted randomly
        # This is specifically the case with the 'chiefs' data - first row of each cell is the GT
        max_score = max(cell_candidate_scores.values())
        choices.append(random.choice([candidate for candidate, score in cell_candidate_scores.items() if score == max_score]))
    return choices


num_trials = 10

headers = ["table", "top1 acc in attainable cells"]#,"cols"]#, "top1 acc", "top 1 acc ignoring ties"]
rows = []
micro_avg_acc = 0
total_num_cells = 0

nic_table_rows_w_gt_max={}

for file in tqdm(sorted(glob.glob(f'{working_path}/dev_predictions/*.csv'))):
# file = "/data02/profiling/contrastive_loss_nn_w_embedding_scores/dev_predictions/14067031_0_559833072073397908.csv"

    file_name = file.split('/')[-1]
    
#     if file_name != 'MOX6MBH5.csv':
#         continue
    
    nic_table_rows_w_gt_max[file_name] = {}
    
    df = pd.read_csv(file).fillna(0)

    scores = df.groupby(['column','row']).apply(lambda g: {cand : g.loc[g.loc[:,"kg_id"]==cand,"siamese_pred"].max() for cand in g.loc[:,"kg_id"].unique()}).to_list()
    cells = df.groupby(['column','row'])["kg_id"].apply(list).to_list()
    cells_gt = [l[0] for l in df.groupby(['column','row'])["GT_kg_id"].apply(list).to_list()]
    cell_idx_to_col = [l[0] for l in df.groupby(['column','row'])["column"].apply(list).to_list()]
    cell_idx_to_row = [l[0] for l in df.groupby(['column','row'])["row"].apply(list).to_list()]

#     avg_f1=0
#     for i in range(num_trials):
#         choices = choose_best_candidates_from_scores(scores)
#         avg_f1 += f1_score(choices, cells_gt, average="micro")
#     avg_f1 /= num_trials
    
    top1_acc = 0
    top1_acc_ignore_ties = 0
    num_no_ties_correct = 0
    num_no_ties = 0
    num_attainable_cells = 0
    l = []
    for cell_idx in range(len(cells_gt)):
        
        row_num = cell_idx_to_row[cell_idx]
        col_num = cell_idx_to_col[cell_idx]
#         if not (row_num ==3 and col_num ==0):
#             continue
        if col_num not in nic_table_rows_w_gt_max[file_name]:
            nic_table_rows_w_gt_max[file_name][col_num] = {}
        nic_table_rows_w_gt_max[file_name][col_num][row_num] = 0
        
        gt = cells_gt[cell_idx]
#         print(gt)
#         print(gt in scores[cell_idx])
#         if gt in scores[cell_idx]:
#             print(scores[cell_idx][gt])
#             print(max(scores[cell_idx].values()))
        max_score_in_row = max(scores[cell_idx].values())
        if gt in [cand for cand, score in scores[cell_idx].items() if score == max_score_in_row]:
            
            nic_table_rows_w_gt_max[file_name][col_num][row_num] = 1
            
            num_cands_tied = len([1 for val in scores[cell_idx].values() if val==max(scores[cell_idx].values())])
            l.append(1 / num_cands_tied)
            top1_acc += (1 / num_cands_tied)
            top1_acc_ignore_ties += 1
        if gt in cells[cell_idx]:
            num_attainable_cells += 1
    micro_avg_acc += top1_acc
    total_num_cells += len(cells_gt)
    top1_attainable_acc = top1_acc / num_attainable_cells
    top1_acc /= len(cells_gt)
    top1_acc_ignore_ties /= len(cells_gt)
#     print(l)
#     print(len(l))
    
    row = [file_name + f"({len(cells_gt)})", top1_acc]
    rows.append(row)
    

avg_row_scores = [0]
for row in rows:
    for i in range(len(avg_row_scores)):
        avg_row_scores[i] += row[i+1]
avg_row_scores = [score / len(rows) for score in avg_row_scores]
rows.append(["Macro Avg"] + avg_row_scores)
rows.append(["Micro Avg"] + [micro_avg_acc / total_num_cells])

rows = [["{:.4f}".format(row[i]) if i > 0 else row[i] for i in range(len(row))] for row in rows]

results_df = pd.DataFrame(data=rows, columns=headers)
display(results_df)
results_df.to_csv(f"{working_path}/best_dev_set_results.csv", index=False)

100%|██████████| 8/8 [00:19<00:00,  2.38s/it]


,table,top1 acc in attainable cells
0,Countries II(35),0.3313
1,Companies(13),0.7308
2,Religious People(151),0.6540
3,Video Games(16),0.6250
4,Movies(75),0.4849
5,Baseball Players I(20),0.6000
6,Baseball Players II(126),0.8320
7,Journals/Magazines(72),0.7986
8,Macro Avg,0.6321
9,Micro Avg,0.6704


In [365]:
results_df = pd.read_csv(f"{working_path}/best_dev_set_results.csv")
macro = results_df.loc[results_df.loc[:,"table"]=="Macro Avg","top1 acc in attainable cells"].item()
micro = results_df.loc[results_df.loc[:,"table"]=="Micro Avg","top1 acc in attainable cells"].item()
print(f"micro: {micro}\nmacro: {macro}")

micro: 0.6704
macro: 0.6321


## Computing top1 accuracy on test set with custom code

In [363]:
# model = TheModelClass(*args, **kwargs)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model = PairwiseNetwork(len(train_features)).to(device=device)

model.load_state_dict(torch.load("{}".format(best_model_path)))
model.eval()
infer_scores(min_max_scaler_path, test_path, test_output_predictions, model)


39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


In [364]:
from sklearn.metrics import f1_score
from tabulate import tabulate
def choose_best_candidates_from_scores(scores):
    choices = []
    for cell_candidate_scores in scores:
        # we need to break ties randomly since the input data may not be formatted randomly
        # This is specifically the case with the 'chiefs' data - first row of each cell is the GT
        max_score = max(cell_candidate_scores.values())
        choices.append(random.choice([candidate for candidate, score in cell_candidate_scores.items() if score == max_score]))
    return choices


num_trials = 10

headers = ["table", "top1 acc in attainable cells"]#,"cols"]#, "top1 acc", "top 1 acc ignoring ties"]
rows = []
micro_avg_acc = 0
total_num_cells = 0

nic_table_rows_w_gt_max={}

for file in tqdm(sorted(glob.glob(f'{working_path}/test_predictions/*.csv'))):
# file = "/data02/profiling/contrastive_loss_nn_w_embedding_scores/dev_predictions/14067031_0_559833072073397908.csv"

    file_name = file.split('/')[-1]
    
#     if file_name != 'MOX6MBH5.csv':
#         continue
    
    nic_table_rows_w_gt_max[file_name] = {}
    
    df = pd.read_csv(file).fillna(0)

    scores = df.groupby(['column','row']).apply(lambda g: {cand : g.loc[g.loc[:,"kg_id"]==cand,"siamese_pred"].max() for cand in g.loc[:,"kg_id"].unique()}).to_list()
    cells = df.groupby(['column','row'])["kg_id"].apply(list).to_list()
    cells_gt = [l[0] for l in df.groupby(['column','row'])["GT_kg_id"].apply(list).to_list()]
    cell_idx_to_col = [l[0] for l in df.groupby(['column','row'])["column"].apply(list).to_list()]
    cell_idx_to_row = [l[0] for l in df.groupby(['column','row'])["row"].apply(list).to_list()]

#     avg_f1=0
#     for i in range(num_trials):
#         choices = choose_best_candidates_from_scores(scores)
#         avg_f1 += f1_score(choices, cells_gt, average="micro")
#     avg_f1 /= num_trials
    
    top1_acc = 0
    top1_acc_ignore_ties = 0
    num_no_ties_correct = 0
    num_no_ties = 0
    num_attainable_cells = 0
    l = []
    for cell_idx in range(len(cells_gt)):
        
        row_num = cell_idx_to_row[cell_idx]
        col_num = cell_idx_to_col[cell_idx]
#         if not (row_num ==3 and col_num ==0):
#             continue
        if col_num not in nic_table_rows_w_gt_max[file_name]:
            nic_table_rows_w_gt_max[file_name][col_num] = {}
        nic_table_rows_w_gt_max[file_name][col_num][row_num] = 0
        
        gt = cells_gt[cell_idx]
#         print(gt)
#         print(gt in scores[cell_idx])
#         if gt in scores[cell_idx]:
#             print(scores[cell_idx][gt])
#             print(max(scores[cell_idx].values()))
        max_score_in_row = max(scores[cell_idx].values())
        if gt in [cand for cand, score in scores[cell_idx].items() if score == max_score_in_row]:
            
            nic_table_rows_w_gt_max[file_name][col_num][row_num] = 1
            
            num_cands_tied = len([1 for val in scores[cell_idx].values() if val==max(scores[cell_idx].values())])
            l.append(1 / num_cands_tied)
            top1_acc += (1 / num_cands_tied)
            top1_acc_ignore_ties += 1
        if gt in cells[cell_idx]:
            num_attainable_cells += 1
    micro_avg_acc += top1_acc
    total_num_cells += len(cells_gt)
    top1_attainable_acc = top1_acc / num_attainable_cells
    top1_acc /= len(cells_gt)
    top1_acc_ignore_ties /= len(cells_gt)
#     print(l)
#     print(len(l))
    
    row = [file_name + f"({len(cells_gt)})", top1_acc]
    rows.append(row)
    

avg_row_scores = [0]
for row in rows:
    for i in range(len(avg_row_scores)):
        avg_row_scores[i] += row[i+1]
avg_row_scores = [score / len(rows) for score in avg_row_scores]
rows.append(["Macro Avg"] + avg_row_scores)
rows.append(["Micro Avg"] + [micro_avg_acc / total_num_cells])

rows = [["{:.4f}".format(row[i]) if i > 0 else row[i] for i in range(len(row))] for row in rows]

results_df = pd.DataFrame(data=rows, columns=headers)
display(results_df)
results_df.to_csv(f"{working_path}/test_set_results.csv", index=False)

100%|██████████| 8/8 [00:19<00:00,  2.38s/it]


,table,top1 acc in attainable cells
0,Countries II(35),0.3313
1,Companies(13),0.7308
2,Religious People(151),0.6540
3,Video Games(16),0.6250
4,Movies(75),0.4849
5,Baseball Players I(20),0.6000
6,Baseball Players II(126),0.8320
7,Journals/Magazines(72),0.7986
8,Macro Avg,0.6321
9,Micro Avg,0.6704


In [365]:
results_df = pd.read_csv(f"{working_path}/test_set_results.csv")
macro = results_df.loc[results_df.loc[:,"table"]=="Macro Avg","top1 acc in attainable cells"].item()
micro = results_df.loc[results_df.loc[:,"table"]=="Micro Avg","top1 acc in attainable cells"].item()
print(f"micro: {micro}\nmacro: {macro}")

micro: 0.6704
macro: 0.6321


In [125]:
# {table_id : sum(rows_correct.values()) for table_id, rows_correct in table_rows_w_gt_max.items()}

{'14067031_0_559833072073397908': 31,
 '14380604_4_3329235705746762392': 11,
 '28086084_0_3127660530989916727': 102,
 '29414811_2_4773219892816395776': 14,
 '39759273_0_1427898308030295194': 68,
 '45073662_0_3179937335063201739': 14,
 '50270082_0_444360818941411589': 113,
 '84575189_0_6365692015941409487': 65}

In [126]:
# {table_id : sum(rows_correct.values()) for table_id, rows_correct in nic_table_rows_w_gt_max.items()}

{'14067031_0_559833072073397908.csv': 31,
 '14380604_4_3329235705746762392.csv': 11,
 '28086084_0_3127660530989916727.csv': 102,
 '29414811_2_4773219892816395776.csv': 14,
 '39759273_0_1427898308030295194.csv': 68,
 '45073662_0_3179937335063201739.csv': 14,
 '50270082_0_444360818941411589.csv': 113,
 '84575189_0_6365692015941409487.csv': 65}

In [107]:
# nic_table_rows_w_gt_max['14380604_4_3329235705746762392.csv']

{0: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 7: 1,
 8: 0,
 9: 1,
 10: 0,
 12: 0,
 13: 1,
 14: 0,
 17: 1}

In [109]:
# table_rows_w_gt_max['14380604_4_3329235705746762392']

{0: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 0,
 12: 1,
 13: 1,
 14: 0,
 17: 1}

In [ ]:
# df = pd.read_csv(f'{working_path}/dev_predictions/14380604_4_3329235705746762392.csv')
# max_val = df.loc[df.loc[:,"row"]==8, "siamese_pred"].max()
# print(max_val)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
# #     display(df)
#     display(df.loc[df.loc[:,"row"]==8])
# #     display(df.loc[df.loc[:,"siamese_pred"]==max_val])